In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import math
from matplotlib.colors import Normalize

In [2]:
loadfile = 'detrended_ssta_OISST.npz'
ssta_dataset = np.load(loadfile)
ssta =  ssta['ssta']
lon2_sst = ssta_dataset['lon2']
lat2_sst = ssta_dataset['lat2']
y = ssta_dataset['y']

ssta = ssta[:,:,(1982<=y)*(y<=2018)]

loadfile = 'slpa_erai.npz'
slpa_dataset = np.load(loadfile)
slpa = slpa_dataset['slpa']
lon2_slp = slpa_dataset['lon2']
lat2_slp = slpa_dataset['lat2']
y = slpa_dataset['y']
m = slpa_dataset['m']
slpa = signal.detrend(slpa)

